In [ ]:
import os
from datetime import datetime
from tqdm import trange
from src.utils.data_util import DataHandlerPI
from src.train_valid_test_step import *
from config import Config as config
from torch.multiprocessing import set_start_method
from src.classifiers.paraphrase_identifier import ParaphraseIdentifier
from src.utils.file_util import *

In [ ]:
# Load model
data_handler = DataHandlerPI()

# Manage and initialize model
# ---------------------------------------------------------------------------------
# Initialize model
epoch_start = 0  #
model = ParaphraseIdentifier(data_handler.config)
model.to(config.DEVICE)

In [3]:
# print out current test model information
print('Adapter Name: {}'.format(config.ADAPTER_NAME))
print('Adapter Split: {}'.format(config.SPLIT))
print('Task Split: {}'.format(config.CLS_TYPE))

Adapter Name: fusion
Adapter Split: random
Task Split: random


In [4]:
model.eval()
bbar = tqdm(enumerate(data_handler.testset_generator),
                ncols=100, leave=False, total=data_handler.config.num_batch_test)

labels, preds = [], []
mc_preds = []
for idx, data in bbar:

    with torch.no_grad():
        # model forward pass to compute loss
        loss, logits = model(data)
    ys = data['labels'].cpu().detach().numpy().tolist()
    labels += ys
    ys_ = torch.argmax(logits, dim=-1).cpu().detach().numpy().tolist()
    preds += ys_
    mc_preds += [1 for _ in ys_]

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def compute_performance(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

In [7]:
eval_res = compute_performance(labels, preds)
eval_res

{'accuracy': 0.9084205518553758,
 'precision': 0.8731635651322233,
 'recall': 0.9339968569931901,
 'f1': 0.9025563148569983}